### Setup

In [1]:
import pandas as pd
import numpy as np
import os
from time import time

In [2]:
#import sys
#sys.path.append(os.path.abspath('..'))#sys.path[0] + '/..') 
from vb_estimators import MultiPipe, LinRegSupreme,LinSVR,RBFSVR,ENet,L1Lars,GBR,HGBR,FlexiblePipe
from vb_helper import VBHelper
from vb_cross_validator import regressor_q_stratified_cv

no daal4py


In [3]:

from warnings import filterwarnings
filterwarnings('ignore')


In [4]:
gridpoints=5
kwargs=dict(
    test_share=0.2,
    cv_folds=10,
    cv_reps=1,
    #cv_groupcount=None,
    cv_strategy=None, # for stratified cv
    random_state=2 # random_state for reproducibility
)
vbhelper=VBHelper(**kwargs)

In [5]:
scorer_list=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'] #cross_validate wants strings
vbhelper.scorer_list=scorer_list

##### Example Dataset

In [6]:
data_path=os.path.join('sample_data','ex1.csv')
df=pd.read_csv(data_path)
all_vars=list(df.columns)
print(all_vars)

['STA_ID', 'LONG', 'LAT', 'OrigHabCode', 'Date', 'THG_Fish', 'YEAR', 'SEASON', 'SUBAREA', 'HABCODE', 'Floc_Depth_ft', 'AFDW_Floc', 'MEHG_Floc', 'THG_floc', 'Tot_Phos_floc', 'Bulk_Dens_Floc', 'Soil_Thickness_FT', 'AFDW_Soil', 'Bulk_Dens_Soil', 'PH_soil', 'SO4_soil', 'MEHG_soil', 'THG_soil', 'Tot_Carbon_Soil_%', 'Tot_Nitrogen_Soil_%', 'Tot_Phos_soil', 'Wat_Depth_ft', 'COND_SW', 'DO_SW', 'TEMP_SW', 'PH_SW', 'TURB_SW', 'REDOX_SW', 'Alk_Phos_SW', 'CHLA_SW', 'CL_SW', 'MEHG_SW', 'NH4_SW', 'NO2_SW', 'NO3_SW', 'SO4_SW', 'Sol_Reac_Phos_SW', 'THG_SW', 'TOC_SW', 'Tot_Nitrogen_SW', 'Tot_Phos_SW', 'REDOX_PW', 'H2S_PW', 'Sol_Reac_Phos_PW', 'MEHG_Peri_AVG', 'THG_epi_peri']


In [7]:
y_name='THG_Fish'
loc_vars=['LAT','LONG']
drop_vars=['Date','OrigHabCode','STA_ID']
drop_vars.extend(loc_vars)
drop_vars.append(y_name)
x_vars=[var for var in all_vars if var not in drop_vars]
X_df=df.loc[:,x_vars]
y_df=df.loc[:,y_name]

In [8]:

shuf=np.arange(y_df.shape[0])
np.random.seed(0)
np.random.shuffle(shuf)
X_df=X_df.iloc[shuf]
y_df=y_df.iloc[shuf]
vbhelper.setData(X_df,y_df)
X_train, X_test, y_train, y_test=vbhelper.train_test_split()

(0, 1, 2, 3) ('YEAR', 'SEASON', 'SUBAREA', 'HABCODE')


In [9]:


est_kwargs=dict(do_prep=False,gridpoints=gridpoints,cat_idx=vbhelper.cat_idx,float_idx=vbhelper.float_idx,bestT=False)
estimator_dict={
    #'gradient-boosting-reg':lambda: GBR(do_prep=False),
    #'lin-reg-supreme':lambda: LinRegSupreme(**est_kwargs),
    #'powXB-least-sq':lambda:FlexiblePipe(**est_kwargs,flex_kwargs={'form':'powXB'}),
    #'expXB-least-sq':lambda:FlexiblePipe(**est_kwargs,flex_kwargs={'form':'expXB'}), #expXB is default
    #'nonlinear-search-least-sq': lambda:FlexiblePipe(**est_kwargs,functional_form_search=True),
    #'robust-powXB-least-sq':lambda:FlexiblePipe(**est_kwargs,flex_kwargs={'form':'powXB','robust':True}),
    #'robust-expXB-least-sq':lambda:FlexiblePipe(**est_kwargs,flex_kwargs={'form':'expXB','robust':True}), #expXB is default
    #'robust-nonlinear-search-least-sq': lambda:FlexiblePipe(**est_kwargs,functional_form_search=True,flex_kwargs={'robust':True}),
    #'histogram-gradient-boosting-reg':HGBR,
    
    
    #'elastic-net':lambda: ENet(**est_kwargs), 
    #'linear-svr-cv':lambda: LinSVR(**est_kwargs), 
    'rbf-svr-cv':lambda: RBFSVR(**est_kwargs), 
    'lassolars':lambda: L1Lars(**est_kwargs),
    
    }
vbhelper.estimator_dict={'multi_pipe':lambda:MultiPipe(pipelist=[(k,v) for k,v in estimator_dict.items()],cat_idx=vbhelper.cat_idx)}
vbhelper.model_dict={key:val() for key,val in vbhelper.estimator_dict.items()} # they will be models once .fit is called

In [10]:
if False:    
    est_kwargs2=est_kwargs
    est_kwargs2['do_prep']=True
    name='lone-lasso'
    vbhelper.estimator_dict[name]=lambda:L1Lars(**est_kwargs2)
    vbhelper.model_dict[name]=vbhelper.estimator_dict[name]()

In [11]:
i=0
for name,est in vbhelper.model_dict.items():
    start=time()
    i+=1;print(name)
    est.fit(X_train,y_train)
    print('train R2:',est.score(X_train,y_train))
    if not X_test is None:print('test R2:',est.score(X_test,y_test))
    end=time()
    print(f'runtime:{(end-start)/60} min.\n')


multi_pipe
train R2: 0.6105981172213643
test R2: 0.45187645582507263
runtime:0.5491766651471456 min.



In [12]:
mp=vbhelper.model_dict['multi_pipe']

In [13]:
fitted_ipipe_dict=mp.build_individual_fitted_pipelines()

In [14]:
fitted_ipipe_dict['lassolars'].score(X_test,y_test)

0.44541109884585783

In [ ]:
vbhelper.runCrossValidate(n_jobs=1)

In [ ]:
#plot cv_yhat

#### graphs and table to summarize results

In [ ]:
vbhelper.buildCVScoreDict()

In [ ]:
vbhelper.viewCVScoreDict()

In [ ]:
vbhelper.plotCVScores(sort=1)

In [ ]:
vbhelper.predictCVYhat()

In [ ]:
vbhelper.plotCVYhat(single_plot=True)